# T-SQL Stored Procedures and Functions

This notebook covers stored procedures, user-defined functions, and advanced T-SQL programming concepts.

**Prerequisites**: Complete the basic setup and joins notebooks first.

## Exercise 1: Variables and Data Types

**Task**: Learn to declare and use variables in T-SQL.

**Concept**: Variables store temporary values and must be declared with specific data types.

In [ ]:
-- Variables and Data Types
DECLARE @EmployeeName NVARCHAR(100);
DECLARE @Salary DECIMAL(10,2);
DECLARE @HireDate DATE;
DECLARE @IsManager BIT;
DECLARE @Counter INT = 1;

-- Assign values
SET @EmployeeName = 'John Smith';
SET @Salary = 75000.00;
SET @HireDate = '2020-01-15';
SET @IsManager = 1;

-- Display values
SELECT 
    @EmployeeName AS Name,
    @Salary AS Salary,
    @HireDate AS HireDate,
    @IsManager AS IsManager,
    @Counter AS Counter;

## Exercise 2: Simple Stored Procedure

**Task**: Create a procedure to get employee information.

**Concept**: Stored procedures are reusable code blocks that can accept parameters.

In [ ]:
-- Create Simple Stored Procedure
CREATE PROCEDURE GetAllEmployees
AS
BEGIN
    SET NOCOUNT ON;
    
    SELECT 
        e.EmployeeID,
        e.FirstName + ' ' + e.LastName AS FullName,
        d.DepartmentName,
        e.Salary,
        e.HireDate
    FROM Employees e
    INNER JOIN Departments d ON e.DepartmentID = d.DepartmentID
    ORDER BY e.LastName, e.FirstName;
END;

-- Execute the procedure
EXEC GetAllEmployees;

## Exercise 3: Procedure with Parameters

**Task**: Create a procedure that accepts input parameters.

**Concept**: Parameters make procedures flexible and reusable with different inputs.

In [ ]:
-- Create Procedure with Input Parameters
CREATE PROCEDURE GetEmployeesByDepartment
    @DepartmentName NVARCHAR(50),
    @MinSalary DECIMAL(10,2) = 0
AS
BEGIN
    SET NOCOUNT ON;
    
    SELECT 
        e.FirstName + ' ' + e.LastName AS EmployeeName,
        e.Email,
        e.Salary,
        e.HireDate,
        DATEDIFF(YEAR, e.HireDate, GETDATE()) AS YearsOfService
    FROM Employees e
    INNER JOIN Departments d ON e.DepartmentID = d.DepartmentID
    WHERE d.DepartmentName = @DepartmentName
      AND e.Salary >= @MinSalary
    ORDER BY e.Salary DESC;
END;

-- Execute with parameters
EXEC GetEmployeesByDepartment @DepartmentName = 'IT', @MinSalary = 60000;

## Exercise 4: Procedure with Output Parameters

**Task**: Create a procedure that returns values through output parameters.

**Concept**: Output parameters allow procedures to return multiple values.

In [ ]:
-- Create Procedure with Output Parameters
CREATE PROCEDURE GetDepartmentStats
    @DepartmentName NVARCHAR(50),
    @EmployeeCount INT OUTPUT,
    @AverageSalary DECIMAL(10,2) OUTPUT,
    @TotalSalaryBudget DECIMAL(12,2) OUTPUT
AS
BEGIN
    SET NOCOUNT ON;
    
    SELECT 
        @EmployeeCount = COUNT(e.EmployeeID),
        @AverageSalary = AVG(e.Salary),
        @TotalSalaryBudget = SUM(e.Salary)
    FROM Employees e
    INNER JOIN Departments d ON e.DepartmentID = d.DepartmentID
    WHERE d.DepartmentName = @DepartmentName;
END;

-- Execute and capture output
DECLARE @EmpCount INT, @AvgSal DECIMAL(10,2), @TotalBudget DECIMAL(12,2);

EXEC GetDepartmentStats 
    @DepartmentName = 'IT',
    @EmployeeCount = @EmpCount OUTPUT,
    @AverageSalary = @AvgSal OUTPUT,
    @TotalSalaryBudget = @TotalBudget OUTPUT;

SELECT @EmpCount AS EmployeeCount, @AvgSal AS AverageSalary, @TotalBudget AS TotalBudget;

## Exercise 5: Scalar Function

**Task**: Create a scalar function that returns a single value.

**Concept**: Scalar functions return one value and can be used in SELECT statements.

In [ ]:
-- Create Scalar Function
CREATE FUNCTION CalculateBonus(@Salary DECIMAL(10,2), @YearsOfService INT)
RETURNS DECIMAL(10,2)
AS
BEGIN
    DECLARE @Bonus DECIMAL(10,2);
    
    SET @Bonus = CASE 
        WHEN @YearsOfService >= 5 THEN @Salary * 0.10
        WHEN @YearsOfService >= 3 THEN @Salary * 0.07
        WHEN @YearsOfService >= 1 THEN @Salary * 0.05
        ELSE @Salary * 0.02
    END;
    
    RETURN @Bonus;
END;

-- Use the function
SELECT 
    FirstName + ' ' + LastName AS EmployeeName,
    Salary,
    DATEDIFF(YEAR, HireDate, GETDATE()) AS YearsOfService,
    dbo.CalculateBonus(Salary, DATEDIFF(YEAR, HireDate, GETDATE())) AS Bonus
FROM Employees
ORDER BY Bonus DESC;

## Exercise 6: Table-Valued Function

**Task**: Create a function that returns a table.

**Concept**: Table-valued functions return a table and can be used like views.

In [ ]:
-- Create Table-Valued Function
CREATE FUNCTION GetEmployeesByDateRange(@StartDate DATE, @EndDate DATE)
RETURNS TABLE
AS
RETURN
(
    SELECT 
        e.EmployeeID,
        e.FirstName + ' ' + e.LastName AS FullName,
        d.DepartmentName,
        e.Salary,
        e.HireDate,
        DATEDIFF(YEAR, e.HireDate, GETDATE()) AS YearsOfService
    FROM Employees e
    INNER JOIN Departments d ON e.DepartmentID = d.DepartmentID
    WHERE e.HireDate BETWEEN @StartDate AND @EndDate
);

-- Use the table-valued function
SELECT * FROM dbo.GetEmployeesByDateRange('2020-01-01', '2022-12-31')
ORDER BY HireDate;

## Exercise 7: Error Handling in Procedures

**Task**: Create a procedure with comprehensive error handling.

**Concept**: TRY-CATCH blocks handle errors gracefully in stored procedures.

In [ ]:
-- Create Procedure with Error Handling
CREATE PROCEDURE AddNewEmployee
    @FirstName NVARCHAR(50),
    @LastName NVARCHAR(50),
    @Email NVARCHAR(100),
    @DepartmentName NVARCHAR(50),
    @Salary DECIMAL(10,2),
    @NewEmployeeID INT OUTPUT
AS
BEGIN
    SET NOCOUNT ON;
    
    DECLARE @DepartmentID INT;
    
    BEGIN TRY
        -- Start transaction
        BEGIN TRANSACTION;
        
        -- Get Department ID
        SELECT @DepartmentID = DepartmentID 
        FROM Departments 
        WHERE DepartmentName = @DepartmentName;
        
        IF @DepartmentID IS NULL
        BEGIN
            THROW 50001, 'Department not found', 1;
        END
        
        -- Insert employee
        INSERT INTO Employees (FirstName, LastName, Email, DepartmentID, Salary)
        VALUES (@FirstName, @LastName, @Email, @DepartmentID, @Salary);
        
        SET @NewEmployeeID = SCOPE_IDENTITY();
        
        -- Commit transaction
        COMMIT TRANSACTION;
        
        PRINT 'Employee added successfully with ID: ' + CAST(@NewEmployeeID AS NVARCHAR(10));
        
    END TRY
    BEGIN CATCH
        -- Rollback transaction
        IF @@TRANCOUNT > 0
            ROLLBACK TRANSACTION;
        
        -- Return error information
        DECLARE @ErrorMessage NVARCHAR(4000) = ERROR_MESSAGE();
        DECLARE @ErrorSeverity INT = ERROR_SEVERITY();
        DECLARE @ErrorState INT = ERROR_STATE();
        
        PRINT 'Error occurred: ' + @ErrorMessage;
        
        -- Re-throw the error
        THROW;
    END CATCH
END;

-- Test the procedure
DECLARE @NewID INT;
EXEC AddNewEmployee 
    @FirstName = 'Alice', 
    @LastName = 'Johnson', 
    @Email = 'alice.johnson@company.com',
    @DepartmentName = 'IT',
    @Salary = 68000,
    @NewEmployeeID = @NewID OUTPUT;

SELECT @NewID AS NewEmployeeID;